I have updated this notebook to modify the wrmsse function  at 29th Mar.  
New wrmsse function for LGBM metric calculate wrmsse only for last 28 days to consider non-zero demand period.  
Please refer comment section. I have commented the detail of my fixing.
(note:I have also remove some variable to reduce the run-time and changed 'objective' in lgbm to 'poisson'.)

This kernel is:  
- Based on [Very fst Model](https://www.kaggle.com/ragnar123/very-fst-model). Thanks [@ragnar123](https://www.kaggle.com/ragnar123).  
- Based on [m5-baseline](https://www.kaggle.com/harupy/m5-baseline). Thank [@harupy](https://www.kaggle.com/harupy).  
to explain the detail of these great notebook by Japanese especially for beginner.  

Additionaly, I have added an relatively efficient evaluation of WRSSE for LGBM metric to these kernel.

## module import

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import lightgbm as lgb
#import dask_xgboost as xgb
#import dask.dataframe as dd
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
import gc
import os
from tqdm import tqdm
from scipy.sparse import csr_matrix

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## functionの定義

reduce_mem_usageは、データのメモリを減らすためにデータ型を変更する関数です。  
('reduce_mem_usage' is a functin which reduce memory usage by changing data type.)
https://qiita.com/hiroyuki_kageyama/items/02865616811022f79754　を参照ください。

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


read_dataはデータの読み込みと, reduce_mem_usageの適用を行う関数  
('read data' is a function to read the files and apply the 'reduce_mem_usage'.)

In [ ]:
def read_data():
    print('Reading files...')
    calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    sales_train_val = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_val.shape[0], sales_train_val.shape[1]))
    
    submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
    
    return calendar, sell_prices, sales_train_val, submission

PandasのdataFrameをきれいに表示する関数
(This function is to diplay a head of Pandas DataFrame.)

In [ ]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [ ]:
calendar, sell_prices, sales_train_val, submission = read_data()

In [ ]:
# 予測期間とitem数の定義 / number of items, and number of prediction period
NUM_ITEMS = sales_train_val.shape[0]  # 30490
DAYS_PRED = submission.shape[1] - 1  # 28

## data加工 (data transform)

### 1.最初にカテゴリ変数の処理(categorical variable)

As [@kaushal2896](https://www.kaggle.com/kaushal2896) suggested in [this comment](https://www.kaggle.com/harupy/m5-baseline#770558), encode the categorical columns before merging to prevent the notebook from crashing even with the full dataset. [@harupy](https://www.kaggle.com/harupy) also use this encoding suggested in [m5-baseline](https://www.kaggle.com/harupy/m5-baseline).  
メモリの効率利用のため, カテゴリ変数をあらかじめLabel encoding.

In [ ]:
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        #not_null = df[col][df[col].notnull()]
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)

    return df


calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales_train_val = encode_categorical(
    sales_train_val, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)


In [ ]:
# sales_train_valからidの詳細部分(itemやdepartmentなどのid)を重複なく一意に取得しておく。(extract a detail of id columns)
product = sales_train_val[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

### 2.sales_train_validationのmelt処理(apply melt to sales_train_validation)  
（時系列の特徴量が作りやすいように, id毎に横に並んだ時系列データを、（id , 時系列）で縦に変換）  
(apply melt to sales_train_validation(time series) to make it easier to treat.)

pandasのmeltを使いdemand(売上数量)を縦に並べる.  
* pandasのmeltは https://qiita.com/ishida330/items/922caa7acb73c1540e28　を参照ください。
* dataの行数が莫大になるので, Kaggle Notebookのmemory制限を考慮し、nrowsで直近365*2日分（2年分）のデータに限定（TODO:環境に応じて期間を変更）

In [ ]:
nrows = 365 * 2 * NUM_ITEMS

In [ ]:
#加工前  
display(sales_train_val.head(5))

to remove data before first non-zero demand date, replace these demand as np.nan.

In [ ]:
d_name = ['d_' + str(i+1) for i in range(1913)]
# d_XX列の値を取得
sales_train_val_values = sales_train_val[d_name].values
print(sales_train_val_values)

In [ ]:
# calculate the start position(first non-zero demand observed date) for each item / 商品の最初の売上日
# 1-1914のdayの数列のうち, 売上が存在しない日を一旦0にし、0を9999に置換。そのうえでminimum numberを計算
# tmp=は各商品について1-1913のベクトルを作成
tmp = np.tile(np.arange(1,1914),(sales_train_val_values.shape[0],1))
print(tmp)

In [ ]:
# 売上が0ではない日が1日以上存在する商品に絞り、売上がない日は０、売上が１つ以上ある場合はd_XXの値が入った行列を作成
df_tmp = ((sales_train_val_values>0) * tmp)
print(df_tmp)

In [ ]:
# 計算手順の補足
# 売上が0の日を9999で埋め（後ほどminを取った際に、売上が発生した最初の日を取得するため）、それ以外の場合はd_XXの値が入った行列を作成
np.where(df_tmp==0,9999,df_tmp)

In [ ]:
# 売上が0の日は9999、それ以外の場合はd_XXの値が入っている中から各最小日を取得し、１引く。商品の売上が初めてあった日-1の値を取得している。
start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1
print(start_no)

In [ ]:
# 計算手順の補足
# 商品が初めて売れた日の逆数を取得。全て1未満の値になる
1/(start_no+1)

In [ ]:
# 計算手順の補足
# 商品が初めて売れた日の逆数で、対角行列を作成する
np.diag(1/(start_no+1))

In [ ]:
# 計算手順の補足
# 商品が初めて売れた日の逆数で対角行列を作成し、1-1914までの日付が商品分並んだ行列tmpと行列の積をとる。
# 売上が初めてあった日以降が1以上の値となっている行列が得られる。
np.dot(np.diag(1/(start_no+1)) , tmp)

In [ ]:
# 売上が初めてあった日以前はTrue, 以降はFalseが入ったflagを作成
flag = np.dot(np.diag(1/(start_no+1)) , tmp)<1
print(flag)

In [ ]:
# 上記で作成したflagを用いて、売上が初めてあった日以前はNaN、それ以降は売上の値が入った行列を作成
sales_train_val_values = np.where(flag,np.nan,sales_train_val_values)
print(sales_train_val_values)

In [ ]:
# 作成した行列を、元のデータフレームに入れる
sales_train_val[d_name] = sales_train_val_values

# 不要な変数は削除しメモリ開放してあげる
del tmp,sales_train_val_values
gc.collect()

In [ ]:
sales_train_val.head()

calculate number of period after first non-zero demand date

In [ ]:
# 売上開始日が最も遅かったものについて、売上開始後に何日分のデータが存在しているか確認
1913-np.max(start_no)

In [ ]:
# meltを利用し、残したい列をid_varsに指定。他の列はdemand列として縦持ちにする。
sales_train_val = pd.melt(sales_train_val,
                                     id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                     var_name = 'day', value_name = 'demand')
sales_train_val.head(3)

In [ ]:
#加工後  
display(sales_train_val.head(5))
print('Melted sales train validation has {} rows and {} columns'.format(sales_train_val.shape[0],
                                                                            sales_train_val.shape[1]))

In [ ]:
# 各商品の直近2年間分のデータに限定
sales_train_val = sales_train_val.iloc[-nrows:,:]

# さらに、demand列がnull"ではない"レコードに限定
sales_train_val = sales_train_val[~sales_train_val.demand.isnull()]

In [ ]:
sales_train_val.head()

### 3.1と同様に予測部分(validation/evaluation部分)のmelt処理し, 学習データと結合する. 出力はdataという変数.

予測部分のsubmission fileを同じくmelt処理し、sales_train_valとつなげる。  
処理の注意点:  
* submission fileの列名を"d_xx"形式に変更する. submission fileで縦に結合されたvalidationとevaluationを一度分割し、それぞれことなる28日間の列名"d_xx"をそれぞれ付与。
* submission fileには, idの詳細（item, department, state等）が無いためidをキーに, sales validationから取得したproductを結合
* test2は、6/1まで不要なため削除

In [ ]:
# seperate test dataframes

# submission fileのidのvalidation部分と, ealuation部分の名前を取得
test1_rows = [row for row in submission['id'] if 'validation' in row]
test2_rows = [row for row in submission['id'] if 'evaluation' in row]

# submission fileのvalidation部分をtest1, ealuation部分をtest2として取得
test1 = submission[submission['id'].isin(test1_rows)]
test2 = submission[submission['id'].isin(test2_rows)]

# test1, test2の列名の"F_X"の箇所をd_XXX"の形式に変更
test1.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + DAYS_PRED)]
test2.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

# test2のidの'_evaluation'を置換
#test1['id'] = test1['id'].str.replace('_validation','')
test2['id'] = test2['id'].str.replace('_evaluation','_validation')


# idをキーにして, idの詳細部分をtest1, test2に結合する.
test1 = test1.merge(product, how = 'left', on = 'id')
test2 = test2.merge(product, how = 'left', on = 'id')

# test1, test2をともにmelt処理する.（売上数量:demandは0）
test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', value_name = 'demand')

test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', value_name = 'demand')

# validation部分と, evaluation部分がわかるようにpartという列を作り、 test1,test2のラベルを付ける。
sales_train_val['part'] = 'train'
test1['part'] = 'test1'
test2['part'] = 'test2'

# sales_train_valとtest1, test2の縦結合.
data = pd.concat([sales_train_val, test1, test2], axis = 0)

# memoryの開放
del sales_train_val, test1, test2

# delete test2 for now(6/1以前は, validation部分のみ提出のため.)
data = data[data['part'] != 'test2']

gc.collect()

### 4.dataにcalendar/sell_pricesを結合

In [ ]:
#calendarの結合
# drop some calendar features(不要な変数の削除:weekdayやwdayなどはdatetime変数から後ほど作成できる。)
calendar.drop(['weekday', 'wday', 'month', 'year'], 
              inplace = True, axis = 1)

# notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)(dayとdをキーにdataに結合)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

# memoryの開放
del  calendar
gc.collect()

#sell priceの結合
# get the sell price data (this feature should be very important)
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

# memoryの開放
del  sell_prices
gc.collect()

In [ ]:
data.head(3)

In [ ]:
data.tail(3)

## 出力

In [ ]:
data.to_pickle("m5_all_data_merged.pickle")